In [ ]:
pip install -U selenium

In [ ]:
!pip install webdriver-manager


In [ ]:
"""Java Version History Scraper"""

#import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run browser in headless mode
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the Wikipedia page
driver.get("https://en.wikipedia.org/wiki/Java_version_history")
time.sleep(2)  # Allow time for the page to load

# Find all tables on the page
tables = driver.find_elements(By.XPATH, "//table[co ntains(@class, 'wikitable')]")

# Loop through each table and extract data
dataframes = []
for index, table in enumerate(tables):
    all_table_rows = table.find_elements(By.XPATH, ".//tr")
    list_of_rows = []
    
    for each_row in all_table_rows:
        list_of_data = [cell.text for cell in each_row.find_elements(By.XPATH, ".//th | .//td")]
        list_of_rows.append(list_of_data)
    
    if len(list_of_rows) > 1:  # Ensure table is not empty
        df = pd.DataFrame(list_of_rows[1:], columns=list_of_rows[0])
        dataframes.append(df)
        df.to_csv(f"java_version_table_{index+1}.csv", index=False)
        print(f"Table {index+1} saved successfully.")

# Close the browser
driver.quit()

print("Scraping completed. CSV files saved.")




In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from webdriver_manager.chrome import ChromeDriverManager

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run browser in headless mode
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the Wikipedia page
driver.get("https://en.wikipedia.org/wiki/Java_version_history")
time.sleep(2)  # Allow time for the page to load

# Find all tables on the page
tables = driver.find_elements(By.XPATH, "//table[contains(@class, 'wikitable')]")

# Prepare a list to hold all dataframes
combined_df = pd.DataFrame()

# Loop through each table and extract data
for table in tables:
    all_table_rows = table.find_elements(By.XPATH, ".//tr")
    list_of_rows = []
    
    for each_row in all_table_rows:
        list_of_data = [cell.text for cell in each_row.find_elements(By.XPATH, ".//th | .//td")]
        list_of_rows.append(list_of_data)

    if len(list_of_rows) > 1:  # Ensure table is not empty
        df = pd.DataFrame(list_of_rows[1:], columns=list_of_rows[0])
        
        # Replace null values with "-"
        df.fillna("-", inplace=True)
        
        # Ensure column names are unique by appending numbers if necessary
        df.columns = [f"{col}_{i}" if df.columns.tolist().count(col) > 1 else col for i, col in enumerate(df.columns)]
        
        # Split rows with multiple values in one cell into separate rows
        for col in df.columns:
            if df[col].dtype == "object":  # Check if column contains string values that can be split
                df[col] = df[col].apply(lambda x: x.split(",") if isinstance(x, str) else [x])
                df = df.explode(col).reset_index(drop=True)

        # Append the current table's data to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save the combined data to a single CSV
combined_df.to_csv("java_version_history_combined.csv", index=False)

# Close the browser
driver.quit()

print("Scraping completed. Data saved to 'java_version_history_combined.csv'.")
